In [1]:
%load_ext autoreload
%autoreload 2
    

# Running MD simulations using nff and ASE

This Jupyter Notebook shows how the `nff` package interfaces with the Atomistic Simulation Environment (ASE). We assume the user went through tutorial `01_training`, so we can load the pretrained models without having to train them again.

As before, importing the dependencies:

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from ase import Atoms
from ase.md.verlet import VelocityVerlet

from nff.md.nve import Dynamics
from nff.data import Dataset
from nff.train import load_model, evaluate
import nff.utils.constants as const
from ase import units
from nff.io import NeuralFF, AtomsBatch

## Loading the relevant data

We reload the dataset and create a `GraphLoader` as we did last time:

In [3]:
dataset = Dataset.from_file('data/dataset.pth.tar')


### Creating Atoms

As before, we can create an `Atoms` object from any element of the dataset. Let's take the first one, for simplicity:

In [4]:
props = dataset[0].copy()
atoms = AtomsBatch(positions=props['nxyz'][:, 1:], 
                   numbers=props['nxyz'][:, 0], 
                   props=props,
                   )

### Creating the ASE calculator

Now we just have to load the ASE calculator from a pretrained model. One way of doing so is through the in-build `from_file` method. You just have to specify the folder where the model was trained and subsequently stored.

In [6]:
nff_ase = NeuralFF.from_file('sandbox/', device=3)

Assigning this calculator to `atoms` is easy:

In [7]:
atoms.set_calculator(nff_ase)

### Configuring the dynamics for the system

In this example, we will run an NVE dynamics simulation. We will use the default parameters there implemented to run a trajectory for an ethanol molecule. The parameters we will specify are the following:

* `T_init`: initial temperature of the simulation
* `time_step`: time step in femtoseconds
* `thermostat`: ASE integrator to use when performing the simulation
* `thermostat_params`: keyword arguments for ase.Integrator class, will be different case-by-case
* `steps`: number of steps to simulate
* `save_frequency`: how often (in steps) save the pose of the molecule in a file
* `nbr_list_update_freq`: how often (in steps) to update the neighbor list (not yet implemented)
* `thermo_filename`: output file for the thermodynamics log
* `traj_filename`: output file for the ASE trajectory file
* `skip`: number of initial frames to skip when recording the trajectory

In [8]:
md_params = {
    'T_init': 450,
    'time_step': 0.5,
#     'thermostat': NoseHoover,   # or Langevin or NPT or NVT or Thermodynamic Integration
#     'thermostat_params': {'timestep': 0.5 * units.fs, "temperature": 120.0 * units.kB,  "ttime": 20.0}
    'thermostat': VelocityVerlet,  
    'thermostat_params': {'timestep': 0.5 * units.fs},
    'steps': 200,
    'save_frequency': 10,
    'nbr_list_update_freq': 20,
    'thermo_filename': 'thermo.log',
    'traj_filename': 'atoms.traj',
    'skip': 0
}

In [10]:
nve = Dynamics(atoms, md_params)
nve.run()

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
0.0000           0.2678      -0.1561       0.4239   364.3

0.0050           0.2688      -0.1239       0.3927   337.6

0.0100           0.2667      -0.2438       0.5105   438.8

0.0150           0.2658      -0.0957       0.3615   310.7

0.0200           0.2652      -0.2693       0.5345   459.5

0.0250           0.2692       0.0289       0.2403   206.6

0.0300           0.2701      -0.1555       0.4256   365.9

0.0350           0.2702       0.0217       0.2485   213.6

0.0400           0.2665      -0.2253       0.4918   422.7

0.0450           0.2668      -0.2337       0.5005   430.2

0.0500           0.2636      -0.2737       0.5374   461.9

0.0550           0.2634      -0.4233       0.6868   590.4

0.0600           0.2674      -0.0992       0.3666   315.1

0.0650           0.2676      -0.2207       0.4883   419.7

0.0700           0.2690      -0.0929       0.3619   311.1

0.0750           0.2659      -0.1738       0.4397   378.0


### Models with directed neighbor lists

The default assumes that you're using SchNet, which uses an undirected neighbor list to save memory. If you're using Painn, DimeNet, or any model with directional information, you will need a directed neighbor list. If you don't specify this then you will get an error. 

While these models automatically make any neighbor list into a directed one when called, the same is not true of periodic boundary conditions. The dataest offsets from the PBC depend on whether the dataset neighbor list is directed or not.

In [11]:
props = dataset[0].copy()
atoms = AtomsBatch(positions=props['nxyz'][:, 1:], 
                   numbers=props['nxyz'][:, 0], 
                   props=props,
                   )
nff_ase = NeuralFF.from_file('sandbox_painn/', device=3)
atoms.set_calculator(nff_ase)

nve = Dynamics(atoms, md_params)
nve.run()

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]


AssertionError: Painn needs a directed neighbor list

If you do then you will be fine!

In [12]:
props = dataset[0].copy()
atoms = AtomsBatch(positions=props['nxyz'][:, 1:], 
                   numbers=props['nxyz'][:, 0], 
                   props=props,
                   directed=True
                   )
nff_ase = NeuralFF.from_file('sandbox_painn/', device=3)
atoms.set_calculator(nff_ase)

nve = Dynamics(atoms, md_params)
nve.run()

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
0.0000           0.2887      -0.1615       0.4502   387.0

0.0050           0.2895      -0.1382       0.4277   367.6

0.0100           0.2885      -0.2719       0.5604   481.7

0.0150           0.2883      -0.1850       0.4734   406.9

0.0200           0.2886      -0.2306       0.5192   446.3

0.0250           0.2885      -0.2713       0.5598   481.2

0.0300           0.2887      -0.1241       0.4129   354.9

0.0350           0.2903      -0.0849       0.3752   322.5

0.0400           0.2901      -0.1346       0.4246   365.0

0.0450           0.2878      -0.1925       0.4803   412.9

0.0500           0.2888      -0.0510       0.3397   292.0

0.0550           0.2875      -0.4088       0.6963   598.5

0.0600           0.2887      -0.2426       0.5313   456.7

0.0650           0.2892      -0.1145       0.4038   347.1

0.0700           0.2902      -0.2418       0.5319   457.2

0.0750           0.2899      -0.2026       0.4925   423.4


The dynamics conserved the energy. The temperature varied throughout the simulation, as expected.

## Visualizing the trajectory

To visualize the trajectory in this Jupyter Notebook, you will have to install the package [nglview](https://github.com/arose/nglview).

In [ ]:
import nglview as nv
from ase.io import Trajectory

Displaying the trajectory:

In [ ]:
%matplotlib notebook

traj = Trajectory('atoms.traj')
nv.show_asetraj(traj)

Looks like the atoms are still together. Visual inspection says that the trajectory is reasonable. Yay for `nff`!